In [8]:
import pandas as pd
data = pd.read_csv("data_dantri_chungkhoan.csv")

In [9]:
print(data.shape)
data.describe()

data.drop_duplicates(subset=["content"],inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)


(580, 4)


Trong Tiếng Anh có rất nhiều từ nối và được sử dụng rất thường xuyên như "and", "the" và "a". Khi thực hiện việc thống kê trên văn bản, những từ này sẽ mang lại rất nhiều nhiễu vì chúng xuất hiện thường xuyên hơn các từ khác. Một số pineline về NLP sẽ gắn cờ chúng là các từ dừng (stop words) - nghĩa là các từ mà bạn có thể sẽ muốn lọc ra trước khi thực hiện bất kỳ các phân tích thống kê nào. Tương tự với Tiếng Việt cũng có rất nhiều stop words, chúng ta cần phải loại bỏ chúng:

In [10]:
# Đọc tệp vietnamesestopword.txt và lưu các từ dừng vào một danh sách
stopwords = []
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

print(f'Number of stopwords: {len(stopwords)}')
print(f'First 10 stopwords: {stopwords[:10]}')


Number of stopwords: 1942
First 10 stopwords: ['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy']


Tiếp theo, chúng ta phân tách dữ liệu văn bản của mình thành đơn vị nhỏ nhất được gọi là "tokens" hoặc "words". Việc phân tích các đoạn văn dài khá khó khăn nên trước tiên chúng ta tách các đoạn văn thành các dòng riêng biệt sau đó các dòng tách thành các từ và xóa stop words:

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
import re
def clean_para(content):
    """Làm sạch văn bản bằng cách loại bỏ dấu câu, chuyển đổi thành chữ thường và loại bỏ stopwords."""
    # Kiểm tra nếu van_ban không phải là chuỗi, thì chuyển thành chuỗi
    if not isinstance(content, str):
        content = ' '.join(content)
    # Loại bỏ dấu câu
    content = re.sub(r'[^\w\s]', '', content)
    # Chuyển đổi thành chữ thường
    content = content.lower()
    # Loại bỏ stopwords
    tu_vung = content.split()
    tu_vung_sach = [tu for tu in tu_vung if tu not in stopwords]
    return ' '.join(tu_vung_sach)

In [14]:
data['content_cleaned'] = data['content'].apply(clean_para)
print(data[['content_cleaned']])

                                       content_cleaned
0    bloomberg billionaires index jensen huang ceo ...
1    giao dịch giằng co phiên phiên chiều 245 áp lự...
2    kết phiên giao dịch 225 mỹ cổ phiếu nvidia 05 ...
3    rung lắc phiên tiếp tục chỉnh đầu phiên chiều ...
4    đỗ quang vinh phó chủ tịch hội đồng quản trị n...
..                                                 ...
569  thị trường giao dịch căng thẳng mệt mỏi hầu đầ...
570  công ty chứng khoán agribank agirseco vnindex ...
571  phiên hôm 73 vnindex tiếp tục diễn biến giá tí...
572  mặc lực chiều dâng đà thu hẹp yếu dần phiên nh...
573  công ty chứng khoán agribank agriseco vnindex ...

[574 rows x 1 columns]


**Xây dựng mô hình LSA**

In [15]:
from sklearn.model_selection import train_test_split
# Chia dữ liệu thành tập huấn luyện, xác thực và tập kiểm tra 70-15-15
text_train, text_val_test = train_test_split(data, test_size=0.3, random_state=42)
text_val, text_test = train_test_split(text_val_test, test_size=0.5, random_state=42)

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Xây dựng pipeline và tìm kiếm tham số tốt nhất
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=clean_para)),
    ('svd', TruncatedSVD())
])
param_grid = {
    'tfidf__max_df': [0.8, 0.9],
    'tfidf__min_df': [1, 2],
    'svd__n_components': [2, 5, 10, 20]
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(text_train['content_cleaned'])


d:\Yến Vy\Jupyternotebook\.conda\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\Yến Vy\Jupyternotebook\.conda\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "d:\Yến Vy\Jupyternotebook\.conda\Lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
d:\Yến Vy\Jupyternotebook\.conda\Lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function clean_para at 0x000001F96B07D760>)),
                                       ('svd', TruncatedSVD())]),
             param_grid={'svd__n_components': [2, 5, 10, 20],
                         'tfidf__max_df': [0.8, 0.9], 'tfidf__min_df': [1, 2]},
             scoring='accuracy')

In [17]:
# Huấn luyện mô hình với tham số tốt nhất
best_params = grid_search.best_params_
model = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=best_params['tfidf__max_df'], min_df=best_params['tfidf__min_df'], tokenizer=clean_para)),
    ('svd', TruncatedSVD(n_components=best_params['svd__n_components']))
])
model.fit(text_train['content_cleaned'])

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8,
                                 tokenizer=<function clean_para at 0x000001F96B07D760>)),
                ('svd', TruncatedSVD())])

***Tóm tắt văn bản***

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
def summarize_text(text, model, train_data): 
    text_vector = model.transform([text])
    
    train_vectors = model.named_steps['tfidf'].transform(train_data['content_cleaned'])
    train_lsa = model.named_steps['svd'].transform(train_vectors) 
    
    # Đảm bảo rằng text_lsa và train_lsa có cùng số lượng thành phần
    n_components = min(text_vector.shape[1], train_lsa.shape[1])
    text_lsa = text_vector[:, :n_components]
    train_lsa = train_lsa[:, :n_components]

    # Tính toán độ tương đồng cosine giữa các câu và văn bản gốc
    similarities = cosine_similarity(text_lsa, train_lsa)[0]
    
    # Xử lý trường hợp không có độ tương đồng
    sentences = nltk.sent_tokenize(text)
    if sentences and similarities.size > 0:
        top_sentence = similarities.argsort()[-1] 
        # Trả về câu đầu tiên nếu chỉ mục không hợp lệ
        return sentences[top_sentence] if top_sentence < len(sentences) else sentences[0] 
    else:
        return "No similar sentences found."

In [19]:
# Sử dụng hàm summarize_text để tạo tóm tắt cho các đoạn văn trong tập dữ liệu kiểm định
text_val['summary'] = text_val['content_cleaned'].apply(lambda x: summarize_text(x, model, text_train)) 
print(text_val[['content_cleaned', 'summary']])

                                       content_cleaned  \
388  pha bẻ lái gay cấn đầu tư đau tim diễn phiên g...   
208  phiên giao dịch hôm đầu tư yếu tim đi khoản rà...   
90   10 cổ phiếu hpg tập đoàn hòa phát bật 31 29900...   
209  2811 trang web công ty cổ vận tải xếp dỡ hải a...   
9    thị trường diễn biến tích cực phiên 165 khoản ...   
..                                                 ...   
11   phiên giao dịch chiều 145 giằng co trạng thái ...   
380  công ty chứng khoán dnse kết thúc phiên giao d...   
532  phiên 293 85 mã giá hose 1 mã trần mã 251 mã 7...   
341  công ty chứng khoán dnse kết phiên 168 vnindex...   
77   ủy ban chứng khoán ubcknn thông báo kiến đơn v...   

                                               summary  
388  pha bẻ lái gay cấn đầu tư đau tim diễn phiên g...  
208  phiên giao dịch hôm đầu tư yếu tim đi khoản rà...  
90   10 cổ phiếu hpg tập đoàn hòa phát bật 31 29900...  
209  2811 trang web công ty cổ vận tải xếp dỡ hải a...  
9    thị trường di

In [20]:
# Sử dụng hàm summarize_text để tạo tóm tắt cho các đoạn văn trong tập dữ liệu kiểm tra
text_test['summary'] = text_test['content_cleaned'].apply(lambda x: summarize_text(x, model, text_train)) 
print(text_test[['content_cleaned', 'summary']])

                                       content_cleaned  \
348  vingroup phát thông cáo vinfast black spade ac...   
369  trạng thái giằng co căng thẳng đầu phiên đầu t...   
84   thị trường giằng co xu hướng lan rộng thời 11h...   
79   14 trang chủ công ty chứng khoán vndirect phát...   
68   phiên giao dịch 263 thị trường cổ phiếu ngành ...   
..                                                 ...   
436  vnindex tiếp tục nỗ lực công phá mốc cản 1100 ...   
76   thị trường chứng khoán phiên 213 tiếp tục hồi ...   
311  cửa phiên 19 sàn chứng khoán nasdaq cổ phiếu v...   
371  công ty chứng khoán dnse kết thúc phiên giao d...   
184  vận động tranh cử thành phố reno mỹ hôm 1712 c...   

                                               summary  
348  vingroup phát thông cáo vinfast black spade ac...  
369  trạng thái giằng co căng thẳng đầu phiên đầu t...  
84   thị trường giằng co xu hướng lan rộng thời 11h...  
79   14 trang chủ công ty chứng khoán vndirect phát...  
68   phiên giao dị

In [21]:
%pip install rouge-score

In [23]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate_summaries(true_summaries, generated_summaries):
    rouge1, rouge2, rougeL = 0, 0, 0
    n = len(true_summaries)
    for true, generated in zip(true_summaries, generated_summaries):
        scores = scorer.score(true, generated)
        rouge1 += scores['rouge1'].precision
        rouge2 += scores['rouge2'].precision
        rougeL += scores['rougeL'].precision
    return {
        'rouge1': rouge1 / n,
        'rouge2': rouge2 / n,
        'rougeL': rougeL / n
    }

#Đánh giá tập val và test
evaluation_scores = evaluate_summaries(text_val['content'], text_val['summary'])
print(f"ROUGE Scores on Validation Set: {evaluation_scores}")

evaluation_scores_test = evaluate_summaries(text_test['content'], text_test['summary'])
print(f"ROUGE Scores on Test Set: {evaluation_scores_test}")

ROUGE Scores on Validation Set: {'rouge1': 0.9454221081585908, 'rouge2': 0.760826612633214, 'rougeL': 0.9421221138983994}
ROUGE Scores on Test Set: {'rouge1': 0.9481295538211897, 'rouge2': 0.7657239537538871, 'rougeL': 0.9442525656683922}
